In [16]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler,RobustScaler
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import balanced_accuracy_score, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV,ParameterGrid,cross_val_score
from sklearn.neural_network import MLPClassifier

In [17]:
# train = pd.read_csv('data_source/train.csv')
# test = pd.read_csv('data_source/test.csv')

# train_robust = train.copy()
# test_robust = test.copy()

x_train = pd.read_csv('data_source/X_train.csv')
y_train = pd.read_csv('data_source/y_train.csv')

x_test = pd.read_csv('data_source/X_test.csv')
y_test = pd.read_csv('data_source/y_test.csv')

x_train_std = x_train.copy()
y_train_std = y_train.copy()

x_train_robust = x_train.copy()
y_train_robust = y_train.copy()

x_test_std = x_test.copy()
y_test_std = y_test.copy()

x_test_robust = x_test.copy()
y_test_robust = y_test.copy()

x_train_std = x_train_std.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_train_std = y_train_std.loc[:,'Action']
x_test_std = x_test_std.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_test_std = y_test_std.loc[:,'Action']

x_train_robust = x_train_robust.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_train_robust = y_train_robust.loc[:,'Action']
x_test_robust = x_test_robust.loc[:,['Destination Port', 'NAT Source Port','Packets', 'Elapsed Time (sec)','Bytes Received']]
y_test_robust = y_test_robust.loc[:,'Action']

In [18]:
scaler = StandardScaler()
robust = RobustScaler()

x_train_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = scaler.fit_transform(x_train_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])
x_test_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = scaler.fit_transform(x_test_std.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])

x_train_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = robust.fit_transform(x_train_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])
x_test_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']] = robust.fit_transform(x_test_robust.loc[:,['Packets','Elapsed Time (sec)','Bytes Received']])


In [4]:
clf = MLPClassifier()
clf.fit(x_train_robust,y_train_robust)
answer = clf.predict(x_test_robust)
print(classification_report(y_test_robust, answer))

              precision    recall  f1-score   support

           0       1.00      0.83      0.91     11292
           1       0.05      0.05      0.05      4496
           2       0.00      0.00      0.00      3856
           3       0.00      1.00      0.01        16

    accuracy                           0.49     19660
   macro avg       0.26      0.47      0.24     19660
weighted avg       0.59      0.49      0.53     19660



In [5]:
parameters = [
    {'activation':['identity','logistic','tanh','relu'],'learning_rate':['constant','invscaling','adaptive']} # ,'solver':['lbfgs','sgd','adam']
]

### Parameter Grid for Robust

In [6]:
best_score = 0
best_grid = ''
for g in ParameterGrid(parameters):
    clf.set_params(**g)
    clf.fit(x_train_robust,y_train_robust)
    answer = clf.predict(x_test_robust)
    f1 = f1_score(y_test_robust,answer,average='macro')

    if f1 > best_score:
        best_score = f1
        best_grid = g
print("F1: %0.5f" % best_score)
print("Grid:", best_grid)

/Users/itthisak/Desktop/Nida/dads-6003-firewall-ml/venv/lib/python3.10/site-packages/sklearn/neural_network/_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


F1: 0.72935
Grid: {'activation': 'tanh', 'learning_rate': 'constant'}


In [9]:
clf = MLPClassifier(activation='tanh',learning_rate='constant')
clf.fit(x_train_robust,y_train_robust)

train_yhat = clf.predict(x_train_robust)
train_f1 = f1_score(y_train_robust,train_yhat,average='macro')

test_yhat = clf.predict(x_test_robust)
test_f1 = f1_score(y_test_robust,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_robust,train_yhat))
print('Test Score\n',classification_report(y_test_robust, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       0.98      0.96      0.97     25830
           1       0.81      0.15      0.25     25165
           2       1.00      1.00      1.00     26245
           3       0.52      0.95      0.67     25744

    accuracy                           0.77    102984
   macro avg       0.83      0.76      0.72    102984
weighted avg       0.83      0.77      0.73    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.89      0.94     11292
           1       0.86      0.16      0.27      4496
           2       0.99      1.00      1.00      3856
           3       0.00      1.00      0.01        16

    accuracy                           0.75     19660
   macro avg       0.71      0.76      0.56     19660
weighted avg       0.97      0.75      0.80     19660

AVG F1-Score Train: 0.7230310256202703
AVG F1-Score Test: 0.5553946416200737


# Parameter Grid for std

In [12]:
best_score = 0
best_grid = ''
for g in ParameterGrid(parameters):
    clf.set_params(**g)
    clf.fit(x_train_std,y_train_std)
    answer = clf.predict(x_test_std)
    f1 = f1_score(y_test_std,answer,average='macro')

    if f1 > best_score:
        best_score = f1
        best_grid = g
print("F1: %0.5f" % best_score)
print("Grid:", best_grid)

F1: 0.74016
Grid: {'activation': 'tanh', 'learning_rate': 'invscaling'}


In [14]:
clf = MLPClassifier(activation='tanh',learning_rate='invscaling')
clf.fit(x_train_std,y_train_std)

train_yhat = clf.predict(x_train_std)
train_f1 = f1_score(y_train_std,train_yhat,average='macro')

test_yhat = clf.predict(x_test_std)
test_f1 = f1_score(y_test_std,test_yhat,average='macro')
print('Train Score\n',classification_report(y_train_std,train_yhat))
print('Test Score\n',classification_report(y_test_std, test_yhat))
print(f"AVG F1-Score Train: {train_f1}\nAVG F1-Score Test: {test_f1}")

Train Score
               precision    recall  f1-score   support

           0       0.81      0.99      0.89     25830
           1       0.56      1.00      0.72     25165
           2       1.00      1.00      1.00     26245
           3       0.37      0.00      0.00     25744

    accuracy                           0.75    102984
   macro avg       0.68      0.75      0.65    102984
weighted avg       0.68      0.75      0.65    102984

Test Score
               precision    recall  f1-score   support

           0       1.00      0.98      0.99     11292
           1       0.96      0.99      0.98      4496
           2       0.99      1.00      0.99      3856
           3       0.00      0.00      0.00        16

    accuracy                           0.99     19660
   macro avg       0.74      0.74      0.74     19660
weighted avg       0.99      0.99      0.99     19660

AVG F1-Score Train: 0.6518364775980865
AVG F1-Score Test: 0.7399979487995794


In [21]:
cross_std = cross_val_score(MLPClassifier(activation='tanh',learning_rate='constant'),x_train_robust,y_train_robust,cv=5)
print(round(np.mean(cross_std),4))

0.7742


In [22]:
cross_std = cross_val_score(MLPClassifier(activation='tanh',learning_rate='invscaling'),x_train_robust,y_train_robust,cv=5)
print(round(np.mean(cross_std),4))

0.7849
